In [1]:
import tensorflow as tf
import torch as tc
from transformers import *
import sqlite3 as sql
from os.path import dirname, join

import datetime as dt
import numpy as np
import pandas.io.sql as psql

from bokeh.io import curdoc
from bokeh.layouts import column, layout
from bokeh.models import ColumnDataSource, Div, Select, Slider, TextInput
from bokeh.plotting import figure

conn = sql.connect("./trigger.sql")
#  query = open(join(dirname(__file__), 'query.sql')).read()
query = "SELECT * From trigger WHERE District is 'Port Loko'"
query = open('query.sql').read()
trigger = psql.read_sql(query, conn)

trigger['Trig_date'] = trigger['Trig_date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))


/home/ollin/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ollin/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ollin/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ollin/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.

In [41]:
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure

from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs

from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_16

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

Loading BokehJS ...

In [17]:
par = trigger.t_q6.str.cat(sep=" ")
par

nlp = pipeline('question-answering')

ans = nlp({
    'question': 'What do you eat ?',
    'context': par
    })


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1634.57it/s]


In [21]:
ans["answer"]

'bush meat'

In [3]:
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show

In [4]:
output_notebook()

Loading BokehJS ...

In [228]:
def get_ans(question, context):
    text_color = '#000000'
    
    ans = nlp({
    'question': question,
    'context': context
    })
    
    p = figure(x_range=(-8, 8), y_range=(-4, 4),
               plot_width = 700, plot_height = 700,
               title = question)
    
    source = ColumnDataSource(data=dict(answer=[ans["answer"]]))
    
    p.rect(0, 0, width=18, height=10, fill_color='#ffffff',
        line_color = 'black', source=source)


    p.text(0, 0, text='answer',
        alpha=0.6667, text_font_size='36pt', text_baseline='middle',
        text_align='center', source=source)
    
    p.xaxis.visible=False
    p.yaxis.visible=False
    
    return p

In [203]:
theq = "What do you eat ?"
par = trigger.t_q6.str.cat(sep=" ")

In [206]:
get_ans(theq, par)


convert squad examples to features: 100%|██████████| 1/1 [00:01<00:00,  1.84s/it]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4766.25it/s]


Figure(id='1931', ...)

In [127]:
def make_dataset(df,active,start,end):
    thewhich = df[df["District"].isin(active)]
    thewhich = thewhich.iloc[thewhich.Trig_date.map(lambda x: (x >= start.date() and x <= end.date())).tolist()]
    par = thewhich.t_q6.str.cat(sep=" ")
    return par

In [214]:
par = make_dataset(trigger,active,init,ends)

In [229]:
theq = "What is ebola ?"

p = get_ans(theq,par)


convert squad examples to features: 100%|██████████| 1/1 [00:02<00:00,  2.09s/it]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 8322.03it/s]


In [230]:
show(p)

In [92]:
df.Trig_date[0] < dt.date(2014,10,12)

False

In [108]:
df.Trig_date.map(lambda x: x >= init.date())

0        True
1       False
2       False
3       False
4       False
        ...  
6748    False
6749    False
6834    False
6835    False
6836    False
Name: Trig_date, Length: 1516, dtype: bool

In [55]:
from bokeh.models import DateRangeSlider

In [70]:
from bokeh.models.widgets import CheckboxGroup# Create the checkbox selection element, available carriers is a  
# list of all airlines in the data
the_districts = CheckboxGroup(labels=trigger.District.unique().tolist(), 
                                  active = [0, 1])

In [201]:
question = TextInput(title="Write your question",value="What is ebola ?")
init = min(trigger.Trig_date)
ends = max(trigger.Trig_date)

range_select = DateRangeSlider(start = init, end = ends, value = (init,ends),
                           step = 1, title = 'Date Span')

In [202]:
controls = WidgetBox(question,the_districts,range_select)
show(controls)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



In [73]:
range_select.on_change('value',update)

In [74]:
active = [the_districts.labels[i] for i in the_districts.active]

In [75]:
active


['Kailahun', 'Bo']

In [ ]:
df[df['A'].isin([3, 6])]

In [13]:
p = figure(plot_width = 700, plot_height = 700, 
                  title = 'Histogram of Arrival Delays by Carrier',
                  x_axis_label = 'Delay (min)', y_axis_label = 'Proportion')



In [148]:
# Update function takes three default parameters
def update(attr, old, new):
    # Get the list of carriers for the graph
    active = [the_districts.labels[i] for i in the_districts.active]

    range_start = range_select.value[0]
    range_end = range_select.value[1]
    
    new_par = make_dataset(trigger,active,range_start,range_end)
  
    # Update the source used the quad glpyhs
    context.data.update(new_par.data)

In [239]:
def modify_plot(doc):
    the_question = TextInput(title="Write your question", value="What is ebola ?")
    #the_question.on_change('active',update)
    
    the_districts = CheckboxGroup(labels=trigger.District.unique().tolist(), 
                                  active = [0, 1])
    the_districts.on_change('active',update)
    
    init = min(trigger.Trig_date)
    ends = max(trigger.Trig_date)
    range_select = DateRangeSlider(start = init, end = ends, value = (init,ends),
                           step = 1, title = 'Date Span')
    range_select.on_change('value',update)
    
    active = [the_districts.labels[i] for i in the_districts.active]
    
    par = make_dataset(trigger,active,range_select.value[0],range_select.value[1])
    
    p = get_ans(theq,par)
    controls = WidgetBox(the_question, the_districts, range_select)
    layout = row(controls, p)
    tab = Panel(child = layout, title = "Question")
    tabs = Tabs(tabs=[tab])
    
    doc.add_root(tabs)

In [180]:
help(TextInput)

Help on class TextInput in module bokeh.models.widgets.inputs:

class TextInput(InputWidget)
 |  TextInput(*args, **kwargs)
 |  
 |  Single-line input widget.
 |  
 |  Method resolution order:
 |      TextInput
 |      InputWidget
 |      bokeh.models.widgets.widget.Widget
 |      bokeh.models.layouts.LayoutDOM
 |      bokeh.model.Model
 |      bokeh.core.has_props.HasProps
 |      bokeh.util.callback_manager.PropertyCallbackManager
 |      bokeh.util.callback_manager.EventCallbackManager
 |      builtins.object
 |  
 |  Data descriptors defined here:
 |  
 |  callback
 |      A callback to run in the browser whenever the user unfocuses the
 |      ``TextInput`` widget by hitting Enter or clicking outside of the text box
 |      area.
 |      
 |      DEPRECATED: use .js_on_change or .on_change with "value" or "value_input"
 |  
 |  placeholder
 |      Placeholder for empty input field.
 |  
 |  value
 |      Initial or entered text value.
 |  
 |  value_input
 |      Initial or entere

In [149]:
the_districts = CheckboxGroup(labels=trigger.District.unique().tolist(), 
                                  active = [0, 1])

In [150]:
the_districts.on_change('active',update)

In [151]:
init = min(trigger.Trig_date)
ends = max(trigger.Trig_date)

In [152]:
range_select = DateRangeSlider(start = init, end = ends, value = (init,ends),
                           step = 1, title = 'Date Span')

In [153]:
range_select.on_change('value',update)

In [173]:
active = [the_districts.labels[i] for i in the_districts.active]

In [174]:
active

['Kailahun', 'Bo']

ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1) content: {'events': [{'kind': 'ModelChanged', 'model': {'type': 'CheckboxGroup', 'id': '1440'}, 'attr': 'active', 'new': [0]}], 'references': []}: NameError("name 'context' is not defined")
ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1) content: {'events': [{'kind': 'ModelChanged', 'model': {'type': 'CheckboxGroup', 'id': '1440'}, 'attr': 'active', 'new': []}], 'references': []}: NameError("name 'context' is not defined")


In [155]:
par = make_dataset(trigger,active,range_select.value[0],range_select.value[1])

In [157]:
p = get_ans(theq,par)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2116.20it/s]


In [160]:
controls = WidgetBox(the_districts, range_select)
layout = row(controls, p)
tab = Panel(child = layout, title = "Question")
tabs = Tabs(tabs=[tab])
    
doc.add_root(tabs)

NameError: name 'doc' is not defined

In [170]:
show(tabs)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



In [240]:
handler = FunctionHandler(modify_plot)

In [241]:
app = Application(handler)

In [242]:
show(app)


convert squad examples to features: 100%|██████████| 1/1 [00:01<00:00,  1.89s/it]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5785.25it/s]
